In [1]:
from libs import generation as gn
from libs import models as md
from libs import settings as st
from libs import utils as ut
from django.db import IntegrityError
import re
import numpy as np

import sqlalchemy as sa
from sqlalchemy import text
import pandas as pd
import datetime as dt
from sqlalchemy import inspect

### Create table

In [2]:
inspector = inspect(st.conn_rotten_engine)
conn = st.conn_rotten_engine.connect()
if len(inspector.get_table_names()):
    md.Base.metadata.create_all(st.conn_rotten_engine)

### Data insertion

#### Genre base

In [3]:
df_genre = gn.dim_genre()
ut.send_data(df_genre, st.conn_rotten_engine, 'dim_genre', 'dbo', True)

### Director base

In [4]:
df_director = gn.dim_director()
ut.send_data(df_director, st.conn_rotten_engine, 'dim_director', 'dbo', True)

### Title base

In [24]:
df_title = gn.dim_title()
ut.send_data(df_title, st.conn_rotten_engine, 'dim_title', 'dbo')

### Language base

In [5]:
df_language = gn.dim_language()
ut.send_data(df_language, st.conn_rotten_engine, 'dim_language', 'dbo')

### Date base

In [3]:
start_date = ''.join(str(gn.generation_df_movies()['release_date_theaters'].min()).split(' ')[0])
end_date = ''.join(str(gn.generation_df_movies()['release_date_theaters'].max()).split(' ')[0])

df_date = gn.dim_date(start_date, end_date)

ut.send_data(df_date, st.conn_rotten_engine, 'dim_date', 'dbo')

### Movies_Genre base

### Movies_Director base

### Raking_Movies base